# Flight Status - Final Project

In [5]:
import os
import shutil
import warnings
import zipfile

import pandas as pd
from scipy import stats
#import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import opendatasets as od
from sklearn.model_selection import train_test_split

from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv
import sqlite3

warnings.filterwarnings('ignore')

## TAS-12 Data Acquisition

### Data Acquisition - Use Kaggle API

[How to use Kaggle API - Step-by-step guide](https://www.geeksforgeeks.org/how-to-download-kaggle-datasets-into-jupyter-notebook/)

Since almost all combined CSV files are larger than 60k records and more than 20 columns the best course of action would be to only use one year data, in this case we are going to use the `Combined_Flights_2022.csv` with the shape `(4078318, 61)` and the `Airlines.csv` for labeling if needed

The **Data Acquisition - Use Kaggle API** section can be re-run only in case we need to access the data from the kaggle API again. After this section we cleaned the data and saved it as CSV in the `/data/processed/` folder, accessed from there, then we'll create a DB with 2 tables `airlines` and `flights`, with the corresponding data and we'll work the data by retrieving it from the DB tables

In [4]:
!kaggle datasets files robikscube/flight-delay-dataset-20182022

name                            size  creationDate         
-----------------------------  -----  -------------------  
Combined_Flights_2018.parquet  215MB  2022-10-07 16:28:11  
Combined_Flights_2021.parquet  232MB  2022-10-07 16:28:11  
Combined_Flights_2019.parquet  294MB  2022-10-07 16:28:11  
Combined_Flights_2018.csv        2GB  2022-10-07 16:28:11  
readme.md                       36KB  2022-10-07 16:28:11  
Combined_Flights_2019.csv        3GB  2022-10-07 16:28:11  
readme.html                     14KB  2022-10-07 16:28:11  
Airlines.csv                    38KB  2022-10-07 16:28:11  
Combined_Flights_2022.parquet  143MB  2022-10-07 16:28:11  
Combined_Flights_2022.csv        1GB  2022-10-07 16:28:11  
Combined_Flights_2020.csv        2GB  2022-10-07 16:28:11  
Combined_Flights_2021.csv        2GB  2022-10-07 16:28:11  
Combined_Flights_2020.parquet  175MB  2022-10-07 16:28:11  


In [5]:
!kaggle datasets download robikscube/flight-delay-dataset-20182022 -f Combined_Flights_2022.csv --path ../data/processed


  0%|          | 0.00/206M [00:00<?, ?B/s]
  0%|          | 1.00M/206M [00:00<00:51, 4.19MB/s]
  1%|          | 2.00M/206M [00:00<00:45, 4.73MB/s]
  1%|▏         | 3.00M/206M [00:00<00:42, 5.02MB/s]
  2%|▏         | 4.00M/206M [00:00<00:43, 4.85MB/s]
  2%|▏         | 5.00M/206M [00:01<00:44, 4.71MB/s]
  3%|▎         | 6.00M/206M [00:01<00:48, 4.32MB/s]
  3%|▎         | 7.00M/206M [00:01<00:47, 4.37MB/s]
  4%|▍         | 8.00M/206M [00:01<00:49, 4.17MB/s]
  4%|▍         | 9.00M/206M [00:02<00:56, 3.66MB/s]
  5%|▍         | 10.0M/206M [00:02<00:55, 3.72MB/s]
  5%|▌         | 11.0M/206M [00:02<00:51, 3.98MB/s]
  6%|▌         | 12.0M/206M [00:03<00:52, 3.90MB/s]
  6%|▋         | 13.0M/206M [00:03<00:50, 4.03MB/s]
  7%|▋         | 14.0M/206M [00:03<00:53, 3.73MB/s]
  7%|▋         | 15.0M/206M [00:03<00:48, 4.11MB/s]
  8%|▊         | 16.0M/206M [00:04<00:47, 4.18MB/s]
  8%|▊         | 17.0M/206M [00:04<00:46, 4.27MB/s]
  9%|▊         | 18.0M/206M [00:04<00:44, 4.43MB/s]
  9%|▉         | 19.

In [6]:
!kaggle datasets download robikscube/flight-delay-dataset-20182022 -f Airlines.csv --path ../data/processed --unzip 

Airlines.csv: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
with zipfile.ZipFile('../data/processed/Combined_Flights_2022.csv.zip', 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall('../data/processed/')

In [8]:
def remove_compressed():
    os.remove('../data/processed/Combined_Flights_2022.csv.zip')

In [9]:
remove_compressed()

### Data Acquisition - Clean and Store Data to CSV

In [7]:
data = pd.read_csv('../data/processed/Combined_Flights_2022.csv')

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4078318 entries, 0 to 4078317
Data columns (total 61 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   FlightDate                               object 
 1   Airline                                  object 
 2   Origin                                   object 
 3   Dest                                     object 
 4   Cancelled                                bool   
 5   Diverted                                 bool   
 6   CRSDepTime                               int64  
 7   DepTime                                  float64
 8   DepDelayMinutes                          float64
 9   DepDelay                                 float64
 10  ArrTime                                  float64
 11  ArrDelayMinutes                          float64
 12  AirTime                                  float64
 13  CRSElapsedTime                           float64
 14  ActualElapsedTime 

In [8]:
data = data.sample(
    n = 500000,
    random_state = 42
)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 104241 to 121958
Data columns (total 21 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Airline                                  500000 non-null  object 
 1   Cancelled                                500000 non-null  int64  
 2   DepTime                                  500000 non-null  object 
 3   DepDelay                                 485439 non-null  float64
 4   ArrTime                                  500000 non-null  object 
 5   AirTime                                  483825 non-null  float64
 6   Distance                                 500000 non-null  float64
 7   Month                                    500000 non-null  int64  
 8   Marketing_Airline_Network                500000 non-null  object 
 9   Operated_or_Branded_Code_Share_Partners  500000 non-null  object 
 10  DOT_ID_Marketing_Airline        

In [14]:
data.describe()

,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,...,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings
count,500000.000000,485442.000000,485439.000000,485439.000000,484973.000000,483825.000000,483825.000000,500000.000000,483825.000000,500000.000000,...,485165.000000,485165.000000,484972.000000,484972.000000,500000.000000,483825.000000,483825.000000,483825.000000,500000.000000,500000.000000
mean,1329.120830,1333.669225,16.100501,13.176115,1457.545614,15.867810,111.126192,141.433184,135.964690,798.916914,...,16.968201,1355.691816,1454.706041,7.882546,1485.733588,7.609537,0.216299,-0.058467,3.666546,0.003784
std,490.729503,505.963408,52.437194,53.445160,543.018210,52.165167,70.018200,71.843747,71.938226,591.845864,...,9.492607,507.930657,537.736098,6.678576,518.277098,55.430586,0.411721,2.497218,2.321489,0.116592
min,1.000000,1.000000,0.000000,-55.000000,1.000000,0.000000,8.000000,5.000000,16.000000,31.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,-91.000000,0.000000,-2.000000,1.000000,0.000000
25%,914.000000,917.000000,0.000000,-5.000000,1046.000000,0.000000,60.000000,89.000000,83.000000,368.000000,...,11.000000,932.000000,1044.000000,4.000000,1102.000000,-14.000000,0.000000,-1.000000,2.000000,0.000000
50%,1320.000000,1325.000000,0.000000,-1.000000,1459.000000,0.000000,94.000000,125.000000,119.000000,646.000000,...,15.000000,1338.000000,1456.000000,6.000000,1512.000000,-5.000000,0.000000,-1.000000,3.000000,0.000000
75%,1735.000000,1743.000000,11.000000,11.000000,1913.000000,10.000000,141.000000,172.000000,167.000000,1035.000000,...,19.000000,1757.000000,1908.000000,9.000000,1920.000000,10.000000,0.000000,0.000000,5.000000,0.000000
max,2359.000000,2400.000000,2650.000000,2650.000000,2400.000000,2678.000000,673.000000,690.000000,711.000000,5095.000000,...,183.000000,2400.000000,2400.000000,251.000000,2359.000000,2678.000000,1.000000,12.000000,11.000000,9.000000


In [15]:
def check_nulls(df):
    # Assuming df is your DataFrame
    # Check for null values in each column
    percentage = 10
    percent = (percentage * len(df)) / 100
    null_counts = df.isnull().sum()

    # Filter columns with null values and print their sum
    columns_with_nulls_ten = null_counts[null_counts > percent]
    columns_with_nulls = null_counts[null_counts > 0]
    if len(columns_with_nulls_ten) > 0:
        for column, count in columns_with_nulls.items():
            print(f"Column '{column}' has {count} null values.")
    else:
        print("The null values in the dataframe don't exceed {percent} values or {percentage}% of the total data".format(percent=percent, percentage=percentage))
        print("Depending on Duplicated values we might want to consider dropping them since that low percentage of null values would hardly make any difference in the EDA or the model creation and prediction")

In [17]:
check_nulls(data)

The null values in the dataframe don't exceed 50000.0 values or 10% of the total data
Depending on Duplicated values we might want to consider dropping them since that low percentage of null values would hardly make any difference in the EDA or the model creation and prediction


In [18]:
# data.duplicated().sum()

In [19]:
def treat_nulls(value):
    if pd.isnull(value):
        return 0
    else:
        return int(value)
    
# Define a function to transform values
def transform_time(value):
    if isinstance(value, int):
        value = str(value)  # Convert integer to string
    value = value.zfill(4)  # Pad with leading zeros if necessary
    if len(value) == 4:
        if int(value) == 2400:
            return '00:00'
        if int(value) < 10:  # For values less than 10
            return f'00:0{value[0]}'
        elif int(value) < 100:  # For values between 10 and 100
            return f'00:{value[:2]}'
        else:  # For values over 100
            return f'{value[:2]}:{value[2:]}'
    else:
        return value[:2] + ':' + value[2:]  # Format as 'HH:MM'

In [20]:
def format_dates():
    data['FlightDate'] = pd.to_datetime(data['FlightDate'])

    # Format time WheelsOff
    data['WheelsOff'] = data['WheelsOff'].apply(treat_nulls)
    data['WheelsOff'] = data['WheelsOff'].apply(transform_time)
    data['WheelsOff'] = pd.to_datetime(data['WheelsOff'], format='%H:%M').dt.time

    # Format time WheelsOn
    data['WheelsOn'] = data['WheelsOn'].apply(treat_nulls)
    data['WheelsOn'] = data['WheelsOn'].apply(transform_time)
    data['WheelsOn'] = pd.to_datetime(data['WheelsOn'], format='%H:%M').dt.time

    # Format time ArrTime
    data['ArrTime'] = data['ArrTime'].apply(treat_nulls)
    data['ArrTime'] = data['ArrTime'].apply(transform_time)
    data['ArrTime'] = pd.to_datetime(data['ArrTime'], format='%H:%M').dt.time


    # Format time DepTime
    data['DepTime'] = data['DepTime'].apply(treat_nulls)
    data['DepTime'] = data['DepTime'].apply(transform_time)
    data['DepTime'] = pd.to_datetime(data['DepTime'], format='%H:%M').dt.time

In [21]:
format_dates()

In [22]:
origins = data[['OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac']]
origins.drop_duplicates(inplace=True)
origins.to_csv('../data/processed/origins.csv', index = False)

In [23]:
destinations = data[['DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac']]
destinations.drop_duplicates(inplace=True)
destinations.to_csv('../data/processed/destinations.csv', index = False)

In [57]:
data.drop(
    columns=[
        'CRSDepTime',
        'ActualElapsedTime',
        'CRSArrTime',
        'OriginAirportSeqID',
        'OriginCityMarketID',
        'Origin',
        'OriginCityName',
        'OriginState',
        'OriginStateFips',
        'OriginStateName',
        'OriginWac',
        'TaxiIn',
        'TaxiOut',
        'ArrDel15',
        'ArrivalDelayGroups',
        'ArrTimeBlk',
        'DistanceGroup',
        'CRSDepTime',
        'DepDel15',
        'DepartureDelayGroups',
        'DepTimeBlk',
        'DestAirportSeqID',
        'DestCityMarketID',
        'Dest',
        'DestCityName',
        'DestState',
        'DestStateFips',
        'DestStateName',
        'DestWac',
        'CRSArrTime',
        'CRSElapsedTime',
        'FlightDate',
        'Diverted',
        'DepDelayMinutes',
        'ArrDelayMinutes',
        'Year',
        'Quarter',
        'DayofMonth',
        'DayOfWeek',
        'Tail_Number',
        'WheelsOn',
        'WheelsOff',
    ], 
    inplace=True
)

In [12]:
colms = data.columns

print(colms)

NameError: name 'data' is not defined

In [59]:
data.to_csv('../data/processed/Combined_Flights_2022.csv', index = False)

In [60]:
data = pd.read_csv('../data/processed/Combined_Flights_2022.csv')
origins = pd.read_csv('../data/processed/origins.csv')
destinations = pd.read_csv('../data/processed/destinations.csv')
airlines = pd.read_csv('../data/processed/Airlines.csv')

In [61]:
data.columns

Index(['Airline', 'Cancelled', 'DepTime', 'DepDelay', 'ArrTime', 'AirTime',
       'Distance', 'Month', 'Marketing_Airline_Network',
       'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline',
       'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline',
       'Operating_Airline', 'DOT_ID_Operating_Airline',
       'IATA_Code_Operating_Airline', 'Flight_Number_Operating_Airline',
       'OriginAirportID', 'DestAirportID', 'ArrDelay', 'DivAirportLandings'],
      dtype='object')

In [62]:
data.head()

,Airline,Cancelled,DepTime,DepDelay,ArrTime,AirTime,Distance,Month,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,...,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,DestAirportID,ArrDelay,DivAirportLandings
0,SkyWest Airlines Inc.,0,18:26:00,-5.0,19:23:00,31.0,125.0,3,UA,UA_CODESHARE,...,UA,5745,OO,20304,OO,5745,10372,11292,-10.0,0
1,SkyWest Airlines Inc.,0,16:05:00,0.0,18:12:00,194.0,1162.0,2,UA,UA_CODESHARE,...,UA,5733,OO,20304,OO,5733,15412,11292,35.0,0
2,American Airlines Inc.,0,17:14:00,-5.0,20:52:00,314.0,2486.0,1,AA,AA,...,AA,205,AA,19805,AA,205,12892,13830,3.0,0
3,Southwest Airlines Co.,0,15:33:00,18.0,23:02:00,251.0,2106.0,1,WN,WN,...,WN,312,WN,19393,WN,312,12889,10821,27.0,0
4,Delta Air Lines Inc.,0,07:09:00,-6.0,08:37:00,73.0,501.0,7,DL,DL,...,DL,698,DL,19790,DL,698,11433,14492,-16.0,0


In [64]:
data.Cancelled.dtype

dtype('int64')

In [65]:
origins.head()

,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac
0,10372,1037205,30372,ASE,"Aspen, CO",CO,8,Colorado,82
1,15412,1541205,35412,TYS,"Knoxville, TN",TN,47,Tennessee,54
2,12892,1289208,32575,LAX,"Los Angeles, CA",CA,6,California,91
3,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85
4,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43


In [66]:
destinations.head()

,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac
0,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82
1,13830,1383002,33830,OGG,"Kahului, HI",HI,15,Hawaii,2
2,10821,1082106,30852,BWI,"Baltimore, MD",MD,24,Maryland,35
3,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37,North Carolina,36
4,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87


In [67]:
airlines.head()

,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


From a bussines perspective point of view I wouldn't impute null values, since, for example, DepTime might be null because the flight might have been cancelled and if we impute or drop that registry we might affect the future predictions or even the hypothesis

### Data Acquisition - Database Creation

In [68]:
load_dotenv()

con = sqlite3.connect("../flight-information.db")
cur = con.cursor()

In [71]:
def map_to_bool(cols):
    mapping = {True: 1, False: 0}
    cols = list(cols)

    for c in cols:
        # Convert boolean values to integers (0 and 1)
        data[c] = data[c].astype(int)
    # for c in cols:
    #     #data[c] = data[c].map(mapping)
    #     data[c] = data[c].astype('int')

In [73]:
# map_to_bool(['Cancelled', 'Diverted'])

In [75]:
# data['Diverted'].unique()

In [77]:
airlines.columns

Index(['Code', 'Description'], dtype='object')

In [78]:
airlines_cols = {
    'Code': 'VARCHAR(50)', 
    'Description': 'VARCHAR(100)'
}

orgs_cols = {
    'OriginAirportID': 'INTEGER',
    'OriginAirportSeqID': 'INTEGER',
    'OriginCityMarketID': 'INTEGER',
    'Origin': 'VARCHAR(10)',
    'OriginCityName': 'VARCHAR(60)',
    'OriginState': 'VARCHAR(10)',
    'OriginStateFips': 'INTEGER',
    'OriginStateName': 'VARCHAR(60)',
    'OriginWac': 'INTEGER',
}
dests_cols = {
    'DestAirportID': 'INTEGER',
    'DestAirportSeqID': 'INTEGER',
    'DestCityMarketID': 'INTEGER',
    'Dest': 'VARCHAR(10)',
    'DestCityName': 'VARCHAR(60)',
    'DestState': 'VARCHAR(10)',
    'DestStateFips': 'INTEGER',
    'DestStateName': 'VARCHAR(60)',
    'DestWac': 'INTEGER',
}

main_cols = {
    'FlightDate': 'DATE',
    'Airline': 'VARCHAR(100)',
    'Cancelled': 'BOOLEAN',
    'Diverted': 'BOOLEAN',
    'DepTime': 'TIME',
    'DepDelayMinutes': 'NUMERIC',
    'DepDelay': 'NUMERIC',
    'ArrTime': 'TIME',
    'ArrDelayMinutes': 'NUMERIC',
    'AirTime': 'NUMERIC',
    'Distance': 'NUMERIC',
    'Year': 'INT',
    'Quarter': 'INT',
    'Month': 'INT',
    'DayofMonth': 'INT',
    'DayOfWeek': 'INT',
    'Marketing_Airline_Network': 'VARCHAR(10)',
    'Operated_or_Branded_Code_Share_Partners': 'VARCHAR(20)',
    'DOT_ID_Marketing_Airline': 'INT',
    'IATA_Code_Marketing_Airline': 'VARCHAR(10)',
    'Flight_Number_Marketing_Airline': 'INT',
    'Operating_Airline': 'VARCHAR(5)',
    'DOT_ID_Operating_Airline': 'INT',
    'IATA_Code_Operating_Airline': 'VARCHAR(5)',
    'Tail_Number': 'VARCHAR(20)', 
    'Flight_Number_Operating_Airline': 'INT',
    'OriginAirportID': 'INT',
    'DestAirportID': 'INT',
    'WheelsOff': 'TIME',
    'WheelsOn': 'TIME',
    'ArrDelay': 'NUMERIC',
    'DivAirportLandings': 'INT' 
}

In [ ]:
main_table = f"""
CREATE TABLE flights(
    {", ".join([f"{col} {dtype}" for col, dtype in main_cols.items()])}
)
"""
origins_table = f"""
CREATE TABLE origins(
    {", ".join([f"{col} {dtype}" for col, dtype in orgs_cols.items()])}
)
"""
dests_table = f"""
CREATE TABLE destinations(
    {", ".join([f"{col} {dtype}" for col, dtype in dests_cols.items()])}
)
"""
airlines_table = f"""
CREATE TABLE airlines(
    {", ".join([f"{col} {dtype}" for col, dtype in airlines_cols.items()])}
)
"""

con.execute(main_table)
con.execute(origins_table)
con.execute(dests_table)
con.execute(airlines_table)





# Iterate over rows in the DataFrame and insert data using parameterized queries
for index, row in data.iterrows():
    # Define the SQL query with placeholders for parameters
    insert_sql = f"""
    INSERT INTO flights ({", ".join(main_cols.keys())}) 
    VALUES ({", ".join(['?' for _ in main_cols.keys()])})
    """
    # Extract values from the row as a tuple
    values = tuple(row[col] for col in main_cols.keys())
    
    # Execute the SQL statement with parameterized values
    cur.execute(insert_sql, values)

# Commit the transaction
con.commit()


for index, row in origins.iterrows():
    # Define the SQL query with placeholders for parameters
    insert_sql = f"""
    INSERT INTO origins ({", ".join(orgs_cols.keys())}) 
    VALUES ({", ".join(['?' for _ in orgs_cols.keys()])})
    """
    # Extract values from the row as a tuple
    values = tuple(row[col] for col in orgs_cols.keys())
    
    # Execute the SQL statement with parameterized values
    cur.execute(insert_sql, values)

# Commit the transaction
con.commit()


for index, row in destinations.iterrows():
    # Define the SQL query with placeholders for parameters
    insert_sql = f"""
    INSERT INTO destinations ({", ".join(dests_cols.keys())}) 
    VALUES ({", ".join(['?' for _ in dests_cols.keys()])})
    """
    # Extract values from the row as a tuple
    values = tuple(row[col] for col in dests_cols.keys())
    
    # Execute the SQL statement with parameterized values
    cur.execute(insert_sql, values)

# Commit the transaction
con.commit()


for index, row in airlines.iterrows():
    # Define the SQL query with placeholders for parameters
    insert_sql = f"""
    INSERT INTO airlines ({", ".join(airlines_cols.keys())}) 
    VALUES ({", ".join(['?' for _ in airlines_cols.keys()])})
    """
    # Extract values from the row as a tuple
    values = tuple(row[col] for col in airlines_cols.keys())
    
    # Execute the SQL statement with parameterized values
    cur.execute(insert_sql, values)

# Commit the transaction
con.commit()

### Data Acquisition - Queries to obtain the data

In [80]:
def get_information():
    sql_query = """
    SELECT 
    flights.*, 
    destinations.DestAirportSeqID,
    destinations.DestCityMarketID,
    destinations.Dest,
    destinations.DestCityName,
    destinations.DestState,
    destinations.DestStateFips,
    destinations.DestStateName,
    destinations.DestWac,
    origins.OriginAirportSeqID,
    origins.OriginCityMarketID,
    origins.Origin,
    origins.OriginCityName,
    origins.OriginState,
    origins.OriginStateFips,
    origins.OriginStateName,
    origins.OriginWac
    FROM flights
    INNER JOIN origins ON flights.OriginAirportID = origins.OriginAirportID
    INNER JOIN destinations ON flights.DestAirportID = destinations.DestAirportID
    """
    return pd.read_sql_query(sql_query, con)

In [81]:
df = get_information()

In [82]:
cur.close()
con.close()

## TAS-19 Descriptive Analysis

First of all, let's talk about concise facts with this data. This dataset has been extracted from the Marketing Carrier On-Time Performance which happened in 2022, the original dataset contains the data of 5 years (2018 to 2022). At the same time this was originally collected from *The Bureau of Transportation Statistics (United States Department of Transportation)*, you can checked on https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGK&QO_fu146_anzr=b0-gvzr and the author or the idea of taking this data originally was made by Rob Mulla(https://www.linkedin.com/in/rob-mulla/) a data scientist with more than 6 years of experience and many contributions made in kaggle(https://www.kaggle.com/robikscube). Therefore our data establishes professional standards to guide the methods and procedures for the collection, processing, storage, and presentation of statistical data which assures us to have reliable data in which we trust.

Our main goal is to make contributions to a status prediction which involves certain airlines in the USA. It's important to mention that our data is precise which ensures us to have the right data in order to make predictions and collaborate with right conclusions.

Once we have gotten enough columns to understand how flights delay goes on, we consider seeing the final columns which we are understanding, *it's appropriate to say there were some columns we've dropped, the reason lies in the fact we only are considering on predict something about flights which have already been made*. Let's take a quick look on the data we own:

In [11]:
data.head(10)

NameError: name 'data' is not defined

### Data Exploration

As we see, there are just 21 columns. These columns are:

In [85]:
print(", ".join(data.columns[:-1]) + " y " + data.columns[-1] + ".")

Airline, Cancelled, DepTime, DepDelay, ArrTime, AirTime, Distance, Month, Marketing_Airline_Network, Operated_or_Branded_Code_Share_Partners, DOT_ID_Marketing_Airline, IATA_Code_Marketing_Airline, Flight_Number_Marketing_Airline, Operating_Airline, DOT_ID_Operating_Airline, IATA_Code_Operating_Airline, Flight_Number_Operating_Airline, OriginAirportID, DestAirportID, ArrDelay y DivAirportLandings.


Let's verify the data type of every column. 

In [117]:
n=1
for col, typ in data.dtypes.items():
    print(f"[{n}] Column '{col}' has data type: '{typ}'")
    n+=1

[1] Column 'Airline' has data type: 'object'
[2] Column 'Cancelled' has data type: 'int32'
[3] Column 'DepTime' has data type: 'object'
[4] Column 'DepDelay' has data type: 'float64'
[5] Column 'ArrTime' has data type: 'object'
[6] Column 'AirTime' has data type: 'float64'
[7] Column 'Distance' has data type: 'float64'
[8] Column 'Month' has data type: 'int64'
[9] Column 'Marketing_Airline_Network' has data type: 'object'
[10] Column 'Operated_or_Branded_Code_Share_Partners' has data type: 'object'
[11] Column 'DOT_ID_Marketing_Airline' has data type: 'int64'
[12] Column 'IATA_Code_Marketing_Airline' has data type: 'object'
[13] Column 'Flight_Number_Marketing_Airline' has data type: 'int64'
[14] Column 'Operating_Airline' has data type: 'object'
[15] Column 'DOT_ID_Operating_Airline' has data type: 'int64'
[16] Column 'IATA_Code_Operating_Airline' has data type: 'object'
[17] Column 'Flight_Number_Operating_Airline' has data type: 'int64'
[18] Column 'OriginAirportID' has data type: '

Now we give a description of what represents every value involved:
- **Airline**: It's just the name of the airline involved.
- **Cancelled**: It's means if a flight was cancelled or not.
- **DepTime**: Gives the exact time at what a flight started (departure).
- **DepDelay**: Mention the amount of time it was delayed the departure.
- **ArrTime**: Indicates the exact moment in which a plane arrives.
- **AirTime**: Show us how much time a plane was on air.
- **Distance**: It has the distance it took going from origin to the destiny.
- **Month**: Says the exact month in which a flight was done.
- **Marketing_Airline_Network:**: Unique Marketing Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **Operated_or_Branded_Code_Partners**: 	Reporting Carrier Operated or Branded Code Share Partners.
-**DOT_ID_Marketing_Airline**: An identification number assigned by US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
- **IATA_Code_Marketing_Airline**: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique.
- **Flight_Number_Marketing_Airline**: It's just the number of flight.
- **Operating_Airline**: Unique Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **DOT_ID_Operating_Airline**: An identification number assigned by US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
- **IATA_Code_Operating_Airline**:Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique.
- **Flight_Number_Operating_Airline**: It's the number of the flight.
- **OriginAirportID**: An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- **DestAirportID**: Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- **ArrDelay**: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
- **DivAirportLandings**: Number of Diverted Airport Landings.

Looking at the description, we observe that there are 4 variables which could be similar, first two are **Operating_Airline** with **IATA_Code_Operating_Airline** and **Marketing_Airline_Network** with **IATA_Code_Marketing_Airline**. Let's take a quick verification:

In [130]:
if data['Operating_Airline'].equals(data['IATA_Code_Operating_Airline']):
    print("The columns 'Operating_Airline' and 'IATA_Code_Operating_Airline' are equal and we must erase one of them.")
else:
    print("The columns 'Operating_Airline' and 'IATA_Code_Operating_Airline' are not equal and we can continue describing these variables.")

if data['Marketing_Airline_Network'].equals(data['IATA_Code_Marketing_Airline']):
    print("The columns 'Marketing_Airline_Network' and 'IATA_Code_Marketing_Airline' are equal and we must erase one of them.")
else:
    print("The columns 'Marketing_Airline_Network' and 'IATA_Code_Marketing_Airline' are not equal and we can continue describing these variables.")
    
    

The columns 'Operating_Airline' and 'IATA_Code_Operating_Airline' are equal and we must erase one of them.
The columns 'Marketing_Airline_Network' and 'IATA_Code_Marketing_Airline' are equal and we must erase one of them.


It's proper to delete some columns on the dataset and just work with one variable of these two in order to do not duplicate values and have enough information. So we proceed to drop **IATA_Code_Operating_Airline**, because the another column has same exact data and is labeled in an easy way. In the same way we drop **IATA_Code_Operating_Airline**.

In [131]:
data.drop(columns=['IATA_Code_Operating_Airline','IATA_Code_Marketing_Airline'], inplace=True)
data.head()

,Airline,Cancelled,DepTime,DepDelay,ArrTime,AirTime,Distance,Month,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,DestAirportID,ArrDelay,DivAirportLandings
0,SkyWest Airlines Inc.,0,18:26:00,-5.0,19:23:00,31.0,125.0,3,UA,UA_CODESHARE,19977,5745,OO,20304,5745,10372,11292,-10.0,0
1,SkyWest Airlines Inc.,0,16:05:00,0.0,18:12:00,194.0,1162.0,2,UA,UA_CODESHARE,19977,5733,OO,20304,5733,15412,11292,35.0,0
2,American Airlines Inc.,0,17:14:00,-5.0,20:52:00,314.0,2486.0,1,AA,AA,19805,205,AA,19805,205,12892,13830,3.0,0
3,Southwest Airlines Co.,0,15:33:00,18.0,23:02:00,251.0,2106.0,1,WN,WN,19393,312,WN,19393,312,12889,10821,27.0,0
4,Delta Air Lines Inc.,0,07:09:00,-6.0,08:37:00,73.0,501.0,7,DL,DL,19790,698,DL,19790,698,11433,14492,-16.0,0


Now let's separate which ones are categorical data types and numerical data types:

In [132]:
cat_list = []
num_list = []

for clm, tp in data.dtypes.items():
    if tp == 'object':  # 'object', means categorical data type
        cat_list.append(clm)
    else:  # According to the table before any other data type is numerical data type
        num_list.append(clm)

print(f"Categorial values are: {cat_list}")
print(f"Numerical values are: {num_list}")

Categorial values are: ['Airline', 'DepTime', 'ArrTime', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'Operating_Airline']
Numerical values are: ['Cancelled', 'DepDelay', 'AirTime', 'Distance', 'Month', 'DOT_ID_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'DOT_ID_Operating_Airline', 'Flight_Number_Operating_Airline', 'OriginAirportID', 'DestAirportID', 'ArrDelay', 'DivAirportLandings']


### Statistical Exploration

Now we are going to use statistical metrics in order to understand the behavior and see if our data contains atypical or missing values. As a general overview we have:

In [133]:
data.describe()

,Cancelled,DepDelay,AirTime,Distance,Month,DOT_ID_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,DestAirportID,ArrDelay,DivAirportLandings
count,500000.000000,485439.000000,483825.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,483825.000000,500000.000000
mean,0.029804,13.176115,111.126192,798.916914,4.083290,19831.380934,2563.063260,19991.688784,2563.081748,12661.447540,12656.907176,7.609537,0.003784
std,0.170046,53.445160,70.018200,591.845864,1.998127,276.433482,1747.493987,376.852386,1747.526268,1523.483014,1523.530683,55.430586,0.116592
min,0.000000,-55.000000,8.000000,31.000000,1.000000,19393.000000,1.000000,19393.000000,1.000000,10135.000000,10135.000000,-91.000000,0.000000
25%,0.000000,-5.000000,60.000000,368.000000,2.000000,19790.000000,1105.000000,19790.000000,1105.000000,11292.000000,11292.000000,-14.000000,0.000000
50%,0.000000,-1.000000,94.000000,646.000000,4.000000,19805.000000,2227.000000,19977.000000,2227.000000,12889.000000,12889.000000,-5.000000,0.000000
75%,0.000000,11.000000,141.000000,1035.000000,6.000000,19977.000000,3879.000000,20378.000000,3879.000000,14027.000000,14027.000000,10.000000,0.000000
max,1.000000,2650.000000,673.000000,5095.000000,7.000000,20436.000000,9677.000000,20500.000000,9677.000000,16869.000000,16869.000000,2678.000000,9.000000


Numerically we can see things right. *Cancelled* values only accept '1's and '0's so it's normal to have these data, *DepDelay* accepts negative values and it doesn't have a ridiculous maximun, *AirTime* goes on accepted minimun and maximun, *Distance* is a parameter that does not change drastically, *Month* has integer values and in a range accepted, *DOT_ID_Marketing_Airline* - *Flight_Number_Marketing_Airline* - *Flight_Number_Operating_Airline* - *DOT_ID_Operating_Airline* - *Flight_Number_Operating_Airline* are data which have codes with a meaningful value ina standard range, at the same time time *ArrDelay1* and *DivAirportLandings* good quality values. There are not evident outliers. 

We take a deep analysis of our variables, taking basic metrics:

In [14]:
def calc_stats(raw_data):
    calc_mean = np.mean(raw_data)
    calc_median = np.median(raw_data)
    calc_mode = stats.mode(raw_data)[0][0]
    calc_std = np.std(raw_data)
    calc_range = np.max(raw_data) - np.min(raw_data)
    
    return calc_mean, calc_median, calc_mode, calc_std, calc_range

def operate(w_data):
    print("Median:", calc_stats(w_data[0]))
    print("Mode:", calc_stats(w_data[1]))
    print("Standard deviation:", calc_stats(w_data[2]))
    print("Range:", calc_stats(w_data[3]))
    return 0

for column_name in data.columns:
    print(f"{column_name} column stats:")
    operate(data[column_name])
    print()

Airline column stats:


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U21'), dtype('<U21')) -> None